In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pwd

Mounted at /content/drive
/content


In [2]:
%cd /content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/

/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master


In [3]:
!pip install -r "/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.4/687.4 kB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirem

In [4]:
!pip install ruamel.yaml
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 25.5 MB/s eta 0:00:00


In [5]:
import os
import math
from tqdm import tqdm
import time
import pyarrow as pa
import pyarrow.parquet as pq


import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px

if os.getcwd().split(os.sep)[-1] == "notebooks":
    os.chdir("../")

# import helper functions coming with this project
import helper.io as io_help
import helper.tracking_data as td_help
import helper.Visualizer as viz_help
import helper.game_models as gm_help

# this is very useful as it makes sure that always all columns and rows of a data frame are displayed
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# **실험실 ☔**

: game 1에 대해서 우선적으로 실행. 추후 game 2, 3에 대하여 진행 예정

# **0) 데이터 전처리**

**0-1) Tracking Data 전처리**

In [6]:
df_match_1 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match1.csv")
df_match_2 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match2.csv")
df_match_3 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match3.csv")

df_prepro_1 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_1.parquet")
df_prepro_2 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_2.parquet")
df_prepro_3 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_3.parquet")

In [7]:
display(df_prepro_1.head())
display(df_prepro_2.head())
display(df_prepro_3.head())

,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,ballInPlay
0,1.0,1.0,0.1,52.47612,28.91763,-1,Ball,52.47612,28.91763,1
1,1.0,1.0,0.1,35.25984,47.03184,1,Home,35.25984,47.03184,1
2,1.0,1.0,0.1,36.39708,35.18136,2,Home,36.39708,35.18136,1
3,1.0,1.0,0.1,33.40116,25.58088,3,Home,33.40116,25.58088,1
4,1.0,1.0,0.1,34.70796,15.30864,4,Home,34.70796,15.30864,1


,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,ballInPlay
0,1.0,1.0,0.1,54.07803,36.27549,-1.0,Ball,54.07803,36.27549,1
1,1.0,1.0,0.1,69.99696,20.5956,1.0,Home,69.99696,20.5956,1
2,1.0,1.0,0.1,73.17216,30.81816,2.0,Home,73.17216,30.81816,1
3,1.0,1.0,0.1,75.47256,40.03632,3.0,Home,75.47256,40.03632,1
4,1.0,1.0,0.1,69.49584,54.90216,4.0,Home,69.49584,54.90216,1


,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,ballInPlay
0,1.0,1.0,0.1,54.123390,35.08146,-1,Ball,54.123390,35.08146,1
1,1.0,1.0,0.1,70.459875,17.71335,1,Home,70.459875,17.71335,1
2,1.0,1.0,0.1,71.815950,33.51735,2,Home,71.815950,33.51735,1
3,1.0,1.0,0.1,73.543545,42.52500,3,Home,73.543545,42.52500,1
4,1.0,1.0,0.1,67.372155,58.04856,4,Home,67.372155,58.04856,1


In [8]:
def preprocessing_tracking_data(df, frame):

  # period	frame	time	xPos	yPos	playerId	team	xPosMetrica	yPosMetrica	ballInPlay
  basic = df[df['frame'] == frame].copy()

  df_list = []
  frame_1_basic = list(basic[['frame', 'session', 'time']].iloc[0])

  # ball 전처리
  ball_basic = list(basic.filter(regex=f'ball').iloc[0]) + [0,0,0,0]
  ball_basic += [-1]
  ball_basic += ['Ball']
  ball_basic = frame_1_basic + ball_basic
  df_list.append(ball_basic)

  # player 전처리
  for i in range(1, 29):
    if i < 10:
      i_basic = list(basic.filter(regex=f'0{i}').iloc[0])
    else:
      i_basic = list(basic.filter(regex=f'{i}').iloc[0])
    i_basic += [i]
    if i <= 14 :
      i_basic += ['Home']
    else:
      i_basic += ['Away']
    frame_df = frame_1_basic + i_basic
    df_list.append(frame_df)

  df_basic = pd.DataFrame(df_list, columns = ['frame', 'period', 'time', 'xPos', 'yPos', 'xvel', 'yvel', 'speed', 'acel', 'playerId', 'team'])
  df_basic.dropna(subset=['xPos', 'yPos'], inplace=True)

  df_basic['xPosMetrica'] = df_basic['xPos']
  df_basic['yPosMetrica'] = df_basic['yPos']
  df_basic = df_basic[['period','frame','time','xPos','yPos','playerId','team','xPosMetrica','yPosMetrica']].copy()

  return df_basic

In [9]:
# BallInPlay 구하기
import preprocessing.cleansing_metrica_data.tracking_data as track_help

def ballInPlay(df, game):

  if game == 3:
    game = 1
  nb_out_frames = 25


  # step 1
  df_match_ball = df[['session', 'frame', 'time', 'ball_x', 'ball_y']].copy()
  df_match_ball.columns = ["Period", "Frame", "Time [s]"] + ["ball_x", "ball_y"]
  df_match_ball = track_help.extract_ball_data(df_match_ball)

  # step 2
  df_match_ball["xPosMetrica"] = df_match_ball["xPos"].copy()
  df_match_ball["yPosMetrica"] = df_match_ball["yPos"].copy()
  df_match_ball = track_help.set_out_ball(df_match_ball, game=game)
  df_frames_out = (
          df_match_ball.groupby("outPeriod").agg(nbFramesOut=("frame", "count")).reset_index()
      )
  df_match_ball = pd.merge(df_match_ball, df_frames_out, on="outPeriod", how="left")
  df_match_ball["ballOutNew"] = np.where(
          (df_match_ball["nbFramesOut"] <= nb_out_frames)
          & (df_match_ball["ballOut"] == 1)
          & (df_match_ball["outOfBounds"] == 0),
          0,
          df_match_ball["ballOut"],
      )

  # step 3
  df_match_ball["xPosTemp"] = df_match_ball["xPos"].copy()
  df_match_ball["yPosTemp"] = df_match_ball["yPos"].copy()
  df_match_ball["xPos"].fillna(method="ffill", inplace=True)
  df_match_ball["yPos"].fillna(method="ffill", inplace=True)
  df_match_ball["xPos"] = np.where(
      df_match_ball["ballOutNew"] != df_match_ball["ballOut"],
      df_match_ball["xPos"],
      df_match_ball["xPosTemp"],
  )
  df_match_ball["yPos"] = np.where(
      df_match_ball["ballOutNew"] != df_match_ball["ballOut"],
      df_match_ball["yPos"],
      df_match_ball["yPosTemp"],
  )

  # delete columns that are not needed any more
  df_match_ball["ballOut"] = df_match_ball["ballOutNew"]
  df_match_ball.drop(["ballOutNew", "xPosTemp", "yPosTemp"], axis=1, inplace=True)

  # update the speed calculation to find frames with issues
  df_match_ball = td_help.add_position_delta(df_match_ball)
  df_match_ball = td_help.add_speed(df_match_ball)

  #sort_values()
  df_match_ball.sort_values(["frame", "playerId"], inplace=True)

  #consider whether the ball is in play rather than whether it is out
  df_match_ball.rename(columns={"ballOut": "ballInPlay"}, inplace=True)
  df_match_ball["ballInPlay"] = 1 - df_match_ball["ballInPlay"]
  df_match_ball.drop(["outOfBounds", "outPeriod"], axis=1, inplace=True)
  df_match_ball = df_match_ball[['period', 'frame', 'time','ballInPlay']]

  return df_match_ball

match_ball_1 = ballInPlay(df_match_1, game=1)
match_ball_2 = ballInPlay(df_match_2, game=2)
match_ball_3 = ballInPlay(df_match_3, game=3)

display(match_ball_1.head())

Do not use this any more but consider using *add_player_velocities*
Do not use this any more but consider using *add_player_velocities*
Do not use this any more but consider using *add_player_velocities*


,period,frame,time,ballInPlay
0,1,1,0.1,1
1,1,2,0.2,1
2,1,3,0.3,1
3,1,4,0.4,1
4,1,5,0.5,1


In [10]:
# a = 1
# for df in [df_match_1, df_match_2, df_match_3]:
#   dflist = []
#   for frame in tqdm(list(df['frame'])):
#     dflist.append(preprocessing_tracking_data(df, frame))

#   df_accum = pd.concat(dflist)
#   df_accum.to_csv(f'/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/preprocessing_{a}.csv')
#   a += 1

# df_prepro_1 = pd.merge(df_prepro_1, match_ball_1, how='inner', on=['frame', 'period', 'time'])
# df_prepro_2 = pd.merge(df_prepro_2, match_ball_1, how='inner', on=['frame', 'period', 'time'])
# df_prepro_3 = pd.merge(df_prepro_3, match_ball_1, how='inner', on=['frame', 'period', 'time'])

In [11]:
# csv to parquet
# Convert Pandas DataFrame to PyArrow Table
# df_prepro = df_prepro_3.copy()
# table = pa.Table.from_pandas(df_prepro)
# pq.write_table(table, '/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_3.parquet')
# table = pq.read_table('/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_3.parquet')
# df = table.to_pandas()

In [12]:
# match 1,2,3 -> tracking_data_game_1/2.parquet 양식으로 맞추기

# a = 1
# for df in [df_match_1, df_match_2, df_match_3]:
#   dflist = []
#   for frame in tqdm(list(df['frame'])):
#     dflist.append(preprocessing_tracking_data(df, frame))

#   df_accum = pd.concat(dflist)
#   df_accum.to_csv(f'/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/preprocessing_{a}.csv')
#   a += 1

# df_prepro_1 = pd.merge(df_prepro_1, match_ball_1, how='inner', on=['frame', 'period', 'time'])
# df_prepro_2 = pd.merge(df_prepro_2, match_ball_1, how='inner', on=['frame', 'period', 'time'])
# df_prepro_3 = pd.merge(df_prepro_3, match_ball_1, how='inner', on=['frame', 'period', 'time'])

**0-2) Event 데이터 전처리**

In [13]:
# event 데이터 전처리
# df_event_game_1 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_1.parquet")
# df_event_match_1 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_1.csv")
# df_event_match_2 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_2.csv")
# df_event_match_3 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_3.csv")

def accurate_list_function(df):
  accurate_1 = df[['type', 'eventName', 'subtype', 'subEventName', 'accurate', 'id', 'matchId']][df['accurate']==1].head(100)
  accurate_list = list(dict.fromkeys(list(accurate_1['eventName'].unique())+ list(accurate_1['subtype'].unique())+ list(accurate_1['subEventName'].unique())))
  accurate_list_2 = []
  for i in accurate_list:
    accurate_list_2.append(i.upper())
  return accurate_list_2


def event_preprocessing(df, game, df_event_game_1):

  game = game

  df['playerId'] = df['from'].str[1:].apply(pd.to_numeric)
  df['toPlayerId'] = df['to'].str[1:].apply(pd.to_numeric)
  df['id'] = [i for i in range(0, len(df))]
  df['matchId'] = game
  df['eventName'] = df['type']
  df['subEventName'] = df['subtype']
  # eventName 변환 & accurate 계산
  df.replace('CHALLENGE', 'Duel', inplace=True)

  accurate_list = [0]
  accurate_list_2 = accurate_list_function(df_event_game_1)
  for i in tqdm(range(1, len(df))):
    i_series = df[['type','subtype','eventName','subEventName']].iloc[i]
    if df.loc[i-1, 'eventName'] == 'Set piece':
      i_series.loc[i, 'eventName'] = 'Set piece'


    accurate = 1
    pass_count = 0
    for i in i_series.to_list():
      if i == 'PASS':
        pass_count += 1
      if i not in accurate_list_2:
        accurate = 0
        break
    if pass_count == 0:
      accurate = 0
    accurate_list.append(accurate)
  df['accurate'] = accurate_list

  df.rename(columns={'start_frame':'startFrame', 'end_frame':'endFrame'}, inplace=True)
  df['startFrame'] = df['startFrame'].apply(pd.to_numeric)
  df['endFrame'] = df['endFrame'].apply(pd.to_numeric)

  return df

# df_event_pro_1 = event_preprocessing(df_event_match_1, 1, df_event_game_1)
# df_event_pro_2 = event_preprocessing(df_event_match_2, 2, df_event_game_1)
# df_event_pro_3 = event_preprocessing(df_event_match_3, 3, df_event_game_1)

# display(df_event_pro_1.head())
# display(df_event_pro_2.head())
# display(df_event_pro_3.head())

In [14]:
# event 데이터를 metrica_data 폴더에 parquet 형식으로 저장

# a = 1
# for i in [df_event_pro_1, df_event_pro_2, df_event_pro_3]:
#   table = pa.Table.from_pandas(i)
#   pq.write_table(table, f'/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_{a}.parquet')
#   a += 1

In [15]:
# 양식 맞춘 후, io_help/td_help 이용해서 데이터 변환
game=1
df_track = io_help.read_tracking_data(game=game, clean=True)
df_track = td_help.add_player_velocities(df_track)
display(df_track.head())

# event의 경우, success 패스를 구하기 위한 최소한의 칼럼만을 구현했으므로, 전처리 그대로 활용
df_event_game_1 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_1.parquet")

,frame,time,period,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,ballInPlay,vx,vy
92,5.0,0.5,1.0,61.113312,21.437568,-1,Ball,61.113312,21.437568,1,0.00000,0.00000
115,6.0,0.6,1.0,61.558380,18.456696,-1,Ball,61.558380,18.456696,1,4.45068,-29.80872
138,7.0,0.7,1.0,61.988436,15.574464,-1,Ball,61.988436,15.574464,1,4.30056,-28.82232
161,8.0,0.8,1.0,62.017164,14.730912,-1,Ball,62.017164,14.730912,1,0.28728,-8.43552
184,9.0,0.9,1.0,61.772004,14.564448,-1,Ball,61.772004,14.564448,1,-2.45160,-1.66464


**1) Success Intended Receiver 예측 및 결과 비교**

**1-1) 데이터에서 Success만을 추출**

In [49]:
# get all passes
df_pass = df_event_game_1[
                (df_event_game_1["eventName"] == "PASS")
                & (df_event_game_1["startFrame"] < df_event_game_1["endFrame"])
                & (df_event_game_1["startFrame"] >= 5)
            ].copy()


# only keep relevant columns
df_pass = df_pass[["id", "matchId", "startFrame", "endFrame", "eventName", "subEventName", "team",
                   "playerId", "toPlayerId", "accurate"]].copy()

# attach the ball position at the start and end frame
df_ball = df_track[df_track["playerId"] == -1][["frame", "xPos", "yPos", "ballInPlay"]].copy()

df_ball.columns = ["startFrame", "xPosStart", "yPosStart", "ballInPlayStart"]
df_pass = pd.merge(df_pass, df_ball, how="left")

df_ball.columns = ["endFrame", "xPosEnd", "yPosEnd", "ballInPlayEnd"]
df_pass = pd.merge(df_pass, df_ball, how="left")
df_pass = df_pass[df_pass["xPosEnd"].notnull()].copy()

# only keep passes for which the ball was in play at the beginning of the pass (i.e. exclude throw-ins)
df_pass = df_pass[df_pass["ballInPlayStart"] == 1].copy()
df_pass.reset_index(inplace=True, drop=True)

# only keep accurate passes
df_pass_acc = df_pass[df_pass["accurate"] == 1].copy()

display(df_pass_acc.head(20))
print(f'정확한 패스의 개수 : {len(df_pass_acc)}/{len(df_pass)}')

,id,matchId,startFrame,endFrame,eventName,subEventName,team,playerId,toPlayerId,accurate,xPosStart,yPosStart,ballInPlayStart,xPosEnd,yPosEnd,ballInPlayEnd
0,3,1,18,25,PASS,PASS,Away,15,19.0,1,59.431968,13.655592,1,48.856716,21.991320,1
1,4,1,31,39,PASS,PASS,Away,19,21.0,1,48.902292,22.805712,1,52.426980,34.024032,1
2,5,1,77,87,PASS,PASS,Away,21,22.0,1,43.330140,52.546752,1,34.839396,69.503976,0
3,15,1,200,215,PASS,HEAD,Home,2,9.0,1,39.545172,48.012696,1,56.666412,42.483312,1
4,16,1,215,223,PASS,PASS,Home,9,10.0,1,56.666412,42.483312,1,54.236952,46.710576,1
5,19,1,306,314,PASS,PASS,Away,16,19.0,1,62.242668,19.353528,1,55.535112,23.789088,1
6,20,1,314,322,PASS,PASS,Away,19,20.0,1,55.535112,23.789088,1,60.800328,33.627024,1
7,21,1,334,353,PASS,PASS,Away,20,22.0,1,57.525120,37.922688,1,47.410812,65.849472,1
8,25,1,454,462,PASS,HEAD,Home,5,6.0,1,34.308792,62.633736,1,33.862752,56.302344,1
9,26,1,462,471,PASS,PASS,Home,6,10.0,1,33.862752,56.302344,1,43.347204,53.624736,1


정확한 패스의 개수 : 671/674


1) startFrame과 'Home'/'Away' 구분해서 prepro table에서 선수들 위치 뽑기 <p>
2) 그것을 토대로 intended receiver 구하기 <p>

In [48]:
df_pass_acc.head(10)

,id,matchId,startFrame,endFrame,eventName,subEventName,team,playerId,toPlayerId,accurate,xPosStart,yPosStart,ballInPlayStart,xPosEnd,yPosEnd,ballInPlayEnd,intended receiver
0,3,1,18,25,PASS,PASS,Away,15,19.0,1,59.431968,13.655592,1,48.856716,21.991320,1,19
1,4,1,31,39,PASS,PASS,Away,19,21.0,1,48.902292,22.805712,1,52.426980,34.024032,1,21
2,5,1,77,87,PASS,PASS,Away,21,22.0,1,43.330140,52.546752,1,34.839396,69.503976,0,22
3,15,1,200,215,PASS,HEAD,Home,2,9.0,1,39.545172,48.012696,1,56.666412,42.483312,1,9
4,16,1,215,223,PASS,PASS,Home,9,10.0,1,56.666412,42.483312,1,54.236952,46.710576,1,10
5,19,1,306,314,PASS,PASS,Away,16,19.0,1,62.242668,19.353528,1,55.535112,23.789088,1,19
6,20,1,314,322,PASS,PASS,Away,19,20.0,1,55.535112,23.789088,1,60.800328,33.627024,1,20
7,21,1,334,353,PASS,PASS,Away,20,22.0,1,57.525120,37.922688,1,47.410812,65.849472,1,22
8,25,1,454,462,PASS,HEAD,Home,5,6.0,1,34.308792,62.633736,1,33.862752,56.302344,1,6
9,26,1,462,471,PASS,PASS,Home,6,10.0,1,33.862752,56.302344,1,43.347204,53.624736,1,10


In [55]:
# Intended Receiver 예측

intended_list = []

for idx, action in df_pass_acc.iterrows():
  # get coordinates of the pass start location, interception point
  # and each potential receiver
  receiver_coo = np.array(
      [
          (o['xPos'], o['yPos'])
          for index, o in df_prepro_1[df_prepro_1['frame'] == action['startFrame']].iterrows()
      ]
  )
  if action['team'] == 'Home':
    receiver_coo = receiver_coo[1:12]
  else:
    receiver_coo = receiver_coo[12:]

  ball_coo = np.array([action.xPosStart, action.yPosStart])
  interception_coo = np.array([action.xPosEnd, action.yPosEnd])

  # compute the distance between the location where the ball was
  # intercepted and each potential receiver
  dist = np.sqrt(
                (receiver_coo[:, 0] - interception_coo[0]) ** 2
                + (receiver_coo[:, 1] - interception_coo[1]) ** 2
            )

  # compute the angle between each potential receiver and the passing line
  a = interception_coo - ball_coo
  b = receiver_coo - ball_coo
  angle = np.arccos(
                np.clip(
                    np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1)), -1, 1
                )
            )
  # if not players are in 20 degrees of the pass line, the intended
  # receiver was probably not in the freeeze frame
  # if np.amin(angle) > 0.35:
  #     continue
  # only consider players in 20 degrees of the pass line
  too_wide = np.where(angle > 0.35)[0]
  dist[too_wide] = np.inf

  # find the most likely intended receiver
  # TODO: you could play around with the weight given to the distance
  # and angle here
  exp_receiver = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))

  if action['team'] == 'Home':
    intended_list.append(exp_receiver+1)
  else:
    intended_list.append(exp_receiver+15)

df_pass_acc['intended_receiver'] = intended_list

<ipython-input-55-54454aaf8f63>:48: RuntimeWarning: invalid value encountered in divide
  exp_receiver = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))


In [58]:
# 기존 데이터와의 Accuracy 실험
accurate_count = 0
for idx, row in df_pass_acc.iterrows():
  if row['toPlayerId'] == row['intended_receiver']:
    accurate_count += 1
  else:
    display(row[['toPlayerId', 'intended_receiver']])

accuracy_rate = round(accurate_count / len(df_pass_acc) * 100, 2)

print(f'Game 1에 대한 intended receiver 정확도 : {accuracy_rate}')

toPlayerId           8.0
intended_receiver      1
Name: 10, dtype: object

toPlayerId           9.0
intended_receiver      8
Name: 11, dtype: object

toPlayerId           2.0
intended_receiver      8
Name: 19, dtype: object

toPlayerId           10.0
intended_receiver       8
Name: 21, dtype: object

toPlayerId           20.0
intended_receiver      22
Name: 23, dtype: object

...

toPlayerId           20.0
intended_receiver      19
Name: 668, dtype: object

toPlayerId           27.0
intended_receiver      24
Name: 669, dtype: object

toPlayerId           12.0
intended_receiver       7
Name: 671, dtype: object

toPlayerId           13.0
intended_receiver      10
Name: 672, dtype: object

Game 1에 대한 intended receiver 정확도 : 41.73


**2) Unsuccess에서 Intended Receiver 성능 비교**

In [ ]:
# Unsuccess 추출

# 2개의 데이터셋 만들기 - Intended Receiver와 기존

In [ ]:
# 실험

In [ ]:
# 결과

**3) 결과 보고**